# Creation of the dataset

This notebook contains the steps for the extraction and...

In [1]:
import os
import pymssql

import pandas as pd

from dotenv import load_dotenv
load_dotenv()

DATASET_CREATION = False

In [22]:
STATUS_PROTESTO = {
    1: "Incluída em lote de remessa",
    2: "Enviada a Protesto",
    3: "Protestada",
    4: "Paga",
    5: "Solicitação de Desistência",
    6: "Solicitação de Cancelamento (Após o Protesto)",
    7: "Solicitação de Autorização de Cancelamento (Dívida Paga ou Parcelada)",
    8: "Cancelada antes do Protesto",
    9: "Cancelada após o Protesto",
    10: "Cancelada por Pagamento",
    11: "Sustada por Ordem Judicial",
    12: "Devolvida por Irregularidade"
}

FINE_CODE_DICT = {
    1: "Ressarcimento",
    2: "Multa",
    3: "Remanejamento",
    4: "Multa Percentual",
    5: "Multa Cominatória"
}

STATUS_DIVIDA_ATIVA = {
    1: "Inscrito em Dívida Ativa",
    2: "Negociado",
    3: "Quitado",
    4: "Exigibilidade Suspensa",
    5: "Cancelado",
    6: "Pagamento em Atraso",
    7: "Remissão",
    8: "Prescrito"
}


In [2]:
def create_raw_dataset():
    queries = {
    "exe_debito": """
        SELECT 
            IdDebito AS exe_debito_iddebito,
            IdProcessoExecucao AS exe_debito_idprocessoexecucao,
            valorOriginalDebito AS exe_debito_valororiginaldebito,
            ValorPago AS exe_debito_valorpago,
            CodigoTipoDebito AS exe_debito_codigotipodebito,
            datainclusao AS exe_debito_datainclusao,
            DataDecisao,
            StatusProtesto AS protesto_status,
            Status_PGE AS pge_status,
            IdProcessoOrigem
        FROM processo.dbo.Exe_Debito
        
    """,
    "exe_debitopessoa": """
        SELECT 
            IDDebitoPessoa AS exe_debitopessoa_iddebitopessoa,
            IDDebito AS exe_debitopessoa_iddebito,
            IDPessoa AS exe_debitopessoa_idpessoa,
            DataInclusao AS exe_debitopessoa_datainclusao
        FROM processo.dbo.Exe_DebitoPessoa
    """,
    "exe_debitoboleto": """
        SELECT 
            IdDebitoBoleto AS exe_debitoboleto_iddebitoboleto,
            IdDebito as exe_debitoboleto_iddebito,
            ValorOriginal AS exe_debitoboleto_valororiginal,
            ValorPago AS exe_debitoboleto_valorpago,
            DataPagamento AS exe_debitoboleto_datapagamento
        FROM processo.dbo.Exe_DebitoBoleto
    """,
    "exe_debito_multadiaria": """
        SELECT 
            IdDebitoMultaDiaria AS exe_debito_multadiaria_iddebitomultadiaria,
            IdDebito as exe_debito_multadiaria_iddebito,
            ValorMultaDiaria AS exe_debito_multadiaria_valormultadiaria,
            DataInicioImputacaoMultaDiaria AS exe_debito_multadiaria_datainicio,
            DataFinalImputacaoMultaDiaria AS exe_debito_multadiaria_datafinal
        FROM processo.dbo.Exe_Debito_MultaDiaria
    """,
    "exe_creditopagamento": """
        SELECT 
            IdCreditoPagamento AS exe_creditopagamento_idcreditopagamento,
            IdDebito as exe_creditopagamento_iddebito,
            ValorCredito AS exe_creditopagamento_valorcredito,
            DataInclusao AS exe_creditopagamento_datainclusao
        FROM processo.dbo.Exe_CreditoPagamento
    """,
    "exe_parcelamento": """
        SELECT 
            IdParcelamento AS exe_parcelamento_idparcelamento,
            IdDebito as exe_parcelamento_iddebito,
            NumeroParcelas AS exe_parcelamento_numeroparcelas,
            SituacaoParcelamento AS exe_parcelamento_situacaoparcelamento,
            DataCancelamentoParcelamento AS exe_parcelamento_datacancelamento,
            DataReabertura AS exe_parcelamento_datareabertura
        FROM processo.dbo.Exe_Parcelamento
    """,
    "protesto_titulosremessa": """
        SELECT 
            IdTituloRemessa AS protesto_titulosremessa_idtituloremessa,
            IdDebito as protesto_titulosremessa_iddebito,
            NumeroProtocoloTitulo AS protesto_titulosremessa_numeroprotocolotitulo,
            DataProtocoloTitulo AS protesto_titulosremessa_dataprotocolotitulo
        FROM processo.dbo.Protesto_TitulosRemessa
    """,
    "pge_processo": """
        SELECT 
            IdProcessoPGE AS pge_processo_idprocessopge,
            IdDebitoExecucao as pge_processo_iddebitoexecucao,
            NumeroProcessoExecucao AS pge_processo_numeroprocessoexecucao,
            AnoProcessoExecucao AS pge_processo_anoprocessoexecucao,
            ValorAtualizadoPGE AS pge_processo_valoratualizadopge,
            ValorPagoPGE AS pge_processo_valorpagopge,
            HomologadoPGE AS pge_processo_homologadopge
        FROM processo.dbo.PGE_Processo
    """,
    "gen_pessoa": """
        SELECT 
            IdPessoa AS pessoa_idpessoa,
            CASE WHEN TipoPessoa = '2' THEN 1 ELSE 0 END AS is_legal_organization
        FROM processo.dbo.GenPessoa
    """,
    "processos": """
        SELECT 
            IdProcesso as processo_idprocesso,
            numero_processo,
            ano_processo
        FROM processo.dbo.Processos
    """,
    "processo_transitojulgado": """
        SELECT 
            numero_processo,
            ano_processo,
            datatransito AS processo_data_transito_julgado
        FROM processo.dbo.Processo_TransitoJulgado
    """,
    "civil_servant_match": """
        SELECT 
    CAST(ano AS INT) AS ano,
    CAST(mes AS INT) AS mes,
    gp.IdPessoa AS pessoa_idpessoa,
    1 AS is_civil_servant
    FROM BdDIP.dbo.vwSiaiPessoalFolhaResumida fr 
    INNER JOIN processo.dbo.GenPessoa gp ON fr.CPF = gp.Documento COLLATE SQL_Latin1_General_CP1_CI_AS
    WHERE ano >= 2023
    GROUP BY ano, mes, gp.IdPessoa

        """
    }

    conn = pymssql.connect(
        server=os.getenv('SQL_SERVER_HOST'),
        port=int(os.getenv('SQL_SERVER_PORT')),
        user=os.getenv('SQL_SERVER_USER'),
        password=os.getenv('SQL_SERVER_PASS'),
        database=os.getenv('SQL_SERVER_DB'),
    )

    exe_debito = pd.read_sql(queries["exe_debito"], conn)
    exe_debitopessoa = pd.read_sql(queries["exe_debitopessoa"], conn)
    exe_debitoboleto = pd.read_sql(queries["exe_debitoboleto"], conn)
    exe_debito_multadiaria = pd.read_sql(queries["exe_debito_multadiaria"], conn)
    exe_creditopagamento = pd.read_sql(queries["exe_creditopagamento"], conn)
    exe_parcelamento = pd.read_sql(queries["exe_parcelamento"], conn)
    protesto_titulosremessa = pd.read_sql(queries["protesto_titulosremessa"], conn)
    pge_processo = pd.read_sql(queries["pge_processo"], conn)
    gen_pessoa = pd.read_sql(queries["gen_pessoa"], conn)
    processos = pd.read_sql(queries["processos"], conn)
    processo_transitojulgado = pd.read_sql(queries["processo_transitojulgado"], conn)
    civil_servant_match = pd.read_sql(queries["civil_servant_match"], conn)


    # Close the connection early
    conn.close()

    # Start with base Exe_Debito
    df = exe_debito.copy()

    # Join with Exe_DebitoPessoa
    df = df.merge(
        exe_debitopessoa,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='exe_debitopessoa_iddebito'
    )

    # Join with Exe_DebitoBoleto
    df = df.merge(
        exe_debitoboleto,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='exe_debitoboleto_iddebito'
    )

    # Join with Exe_Debito_MultaDiaria
    df = df.merge(
        exe_debito_multadiaria,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='exe_debito_multadiaria_iddebito'
    )

    # Join with Exe_CreditoPagamento
    df = df.merge(
        exe_creditopagamento,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='exe_creditopagamento_iddebito'
    )

    # Join with Exe_Parcelamento
    df = df.merge(
        exe_parcelamento,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='exe_parcelamento_iddebito'
    )

    # Join with Protesto_TitulosRemessa
    df = df.merge(
        protesto_titulosremessa,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='protesto_titulosremessa_iddebito'
    )

    # Join with PGE_Processo
    df = df.merge(
        pge_processo,
        how='left',
        left_on='exe_debito_iddebito',
        right_on='pge_processo_iddebitoexecucao'
    )

    # Join with GenPessoa
    df = df.merge(
        gen_pessoa,
        how='left',
        left_on='exe_debitopessoa_idpessoa',
        right_on='pessoa_idpessoa'
    )

    # Join with Processos
    df = df.merge(
        processos,
        how='left',
        left_on='IdProcessoOrigem',
        right_on='processo_idprocesso'
    )

    # Join with Processo_TransitoJulgado
    df = df.merge(
        processo_transitojulgado,
        how='left',
        on=['numero_processo', 'ano_processo']
    )

    # Extract year/month for civil servant matching
    df['ano'] = pd.to_datetime(df['DataDecisao'], errors='coerce').dt.year
    df['mes'] = pd.to_datetime(df['DataDecisao'], errors='coerce').dt.month

    # Join with civil servant match
    df = df.merge(
        civil_servant_match,
        how='left',
        on=['ano', 'mes', 'pessoa_idpessoa']
    )

    # Fill missing with 0
    df['is_civil_servant'] = df['is_civil_servant'].fillna(0).astype(int)

    df.to_csv('../data/private/raw/tcern_debtors_dataset.csv', index=False)

In [3]:
if DATASET_CREATION:
    create_raw_dataset()

In [ ]:
id_columns_to_load_as_int = [
    'exe_debito_iddebito',
    'exe_debito_idprocessoexecucao',
    'exe_debitopessoa_iddebitopessoa',
    'exe_debitopessoa_idpessoa',
    'exe_debitoboleto_iddebitoboleto',
    'exe_debito_multadiaria_iddebitomultadiaria',
    'exe_creditopagamento_idcreditopagamento',
    'exe_parcelamento_idparcelamento',
    'protesto_titulosremessa_idtituloremessa',
    'pge_processo_idprocessopge',
    'protesto_titulosremessa_numeroprotocolotitulo'
]
dtype_map = {col: 'Int64' for col in id_columns_to_load_as_int}
df = pd.read_csv('../data/public/raw/tcern_debtors_dataset.csv', dtype=dtype_map)


/tmp/ipykernel_39618/2276906106.py:15: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/public/processed/tcern_debtors_dataset.csv', dtype=dtype_map)


In [6]:
len(df)


71974

In [7]:
df.head()

,exe_debito_iddebito,exe_debito_idprocessoexecucao,exe_debito_valororiginaldebito,exe_debito_valorpago,exe_debito_codigotipodebito,exe_debito_datainclusao,DataDecisao,protesto_status,pge_status,IdProcessoOrigem,...,pge_processo_homologadopge,pessoa_idpessoa,is_legal_organization,processo_idprocesso,numero_processo,ano_processo,processo_data_transito_julgado,ano,mes,is_civil_servant
0,2,419587,13286.0,NaN,1,2014-11-04 08:18:28.330,2013-03-14,NaN,NaN,232568,...,NaN,3678.0,0.0,232568,9767,2007,2013-08-15,2013.0,3.0,0
1,2,419587,13286.0,NaN,1,2014-11-04 08:18:28.330,2013-03-14,NaN,NaN,232568,...,NaN,3678.0,0.0,232568,9767,2007,2013-08-15,2013.0,3.0,0
2,3,419587,2000.0,3393.88,2,2014-11-04 08:19:54.557,2013-03-14,NaN,True,232568,...,NaN,3678.0,0.0,232568,9767,2007,2013-08-15,2013.0,3.0,0
3,3,419587,2000.0,3393.88,2,2014-11-04 08:19:54.557,2013-03-14,NaN,True,232568,...,NaN,3678.0,0.0,232568,9767,2007,2013-08-15,2013.0,3.0,0
4,3,419587,2000.0,3393.88,2,2014-11-04 08:19:54.557,2013-03-14,NaN,True,232568,...,NaN,3678.0,0.0,232568,9767,2007,2013-08-15,2013.0,3.0,0


# Preprocessing

In [8]:
df['exe_parcelamento_situacaoparcelamento'].fillna(0, inplace=True)


/tmp/ipykernel_39618/3855748047.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['exe_parcelamento_situacaoparcelamento'].fillna(0, inplace=True)


In [9]:
df['exe_parcelamento_datacancelamento'].isna().sum()

np.int64(47026)

# Features for Clustering Debtor Profiles

Financial Features
- `total_debt_value`: Total value of all debts.
- `avg_parcel_value`: Average value of parcelas (installments).
- `max_debt_value`: Maximum single debt value.
- `total_fines_only`: Sum of multas (fines).
- `total_ressarcimento_only`: Sum of ressarcimentos (reimbursements).
- `total_paid_amount`: Total amount already paid.
- `percent_paid_amount`: `total_paid_amount`/`total_debt_value`.

Installment Features
- `n_installments`: Number of signed installment agreements.
- `n_broken_installments`: Count of broken agreements.
- `n_reopened_installments`: Count of reopened agreements.
- `perc_broken_installments`: `n_broken_installments`/`n_installments`
- `perc_failed_installments`: (`n_broken_installments` + `n_reopened_installments`)/`n_installments`

Legal Process Features
- `n_processos`: Number of TCE processes linked to the debtor.
- `n_transitado_em_julgado`: Finalized (unappealable) decisions.
- `total_process_value`: Total value across processes.
- `n_protests`: Number of notary protest actions.
- `n_failed_protests`: Number of failed notary protest actions.
- `perc_failed_protests`: `n_failed_protests`/`n_protests`
- `n_pge_handovers`: Number of PGE (Procuradoria Geral do Estado) handovers.

Debtor Profile Features
- `is_civil_servant`: Employment in public service at the time of the fine decision.

In [ ]:


df['exe_debito_tipodebito'] = df.exe_debito_codigotipodebito.apply(lambda x: FINE_CODE_DICT.get(x, x))
df['is_multa'] = df.exe_debito_tipodebito.apply(lambda x: 1 if x in ['Multa', 'Multa Percentual', 'Multa Cominatória'] else 0)
df['is_ressarcimento'] = df.exe_debito_tipodebito.apply(lambda x: 1 if x in ['Ressarcimento', 'Remanejamento'] else 0)


In [11]:
 # Group by debtor 
grouped = df.groupby('exe_debitopessoa_idpessoa')

In [12]:
# Financial Features
features_df = pd.DataFrame()
features_df['total_debt_value'] = grouped['exe_debito_valororiginaldebito'].sum()
features_df['avg_parcel_value'] = grouped['exe_debitoboleto_valororiginal'].mean()
features_df['max_debt_value'] = grouped['exe_debito_valororiginaldebito'].max()
# Total fines
features_df['total_fines_only'] = grouped.apply(lambda g: g.loc[g['is_multa'] == 1, 'exe_debito_valororiginaldebito'].sum())
# Total devolutions
features_df['total_ressarcimento_only'] = grouped.apply(lambda g: g.loc[g['is_ressarcimento'] == 1, 'exe_debito_valororiginaldebito'].sum())

features_df['has_daily_fine'] = (grouped['exe_debito_multadiaria_iddebitomultadiaria'].count() > 0).astype(int)

features_df['total_paid_amount'] = grouped['exe_debito_valorpago'].sum()

features_df['percent_paid_amount'] = features_df['total_paid_amount'] / features_df['total_debt_value']

/tmp/ipykernel_39618/3194483842.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features_df['total_fines_only'] = grouped.apply(lambda g: g.loc[g['is_multa'] == 1, 'exe_debito_valororiginaldebito'].sum())
/tmp/ipykernel_39618/3194483842.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features_df['total_ressarcimento_only'] = grouped.apply(lambda g: g.loc[g['is_ressarcimento'] == 1, 'exe_debito_valor

In [13]:
# Temporal Features
features_df['n_installments'] = grouped['exe_parcelamento_idparcelamento'].nunique()

features_df['n_broken_installments'] = grouped['exe_parcelamento_datacancelamento'].apply(lambda x: x.notna().sum())
features_df['n_broken_installments'] = features_df['n_broken_installments'].fillna(0)

features_df['n_reopened_installments'] = grouped['exe_parcelamento_datareabertura'].apply(lambda x: x.notna().sum())
features_df['n_reopened_installments'] = features_df['n_reopened_installments'].fillna(0)

features_df['perc_broken_installments'] = features_df['n_broken_installments'] / features_df['n_installments']
features_df['perc_broken_installments'] = features_df['perc_broken_installments'].fillna(0)
features_df['perc_failed_installments'] = (features_df['n_reopened_installments']+features_df['n_broken_installments']) / features_df['n_installments']
features_df['perc_failed_installments'] = features_df['perc_failed_installments'].fillna(0)

In [14]:
df.columns

Index(['exe_debito_iddebito', 'exe_debito_idprocessoexecucao',
       'exe_debito_valororiginaldebito', 'exe_debito_valorpago',
       'exe_debito_codigotipodebito', 'exe_debito_datainclusao', 'DataDecisao',
       'protesto_status', 'pge_status', 'IdProcessoOrigem',
       'exe_debitopessoa_iddebitopessoa', 'exe_debitopessoa_iddebito',
       'exe_debitopessoa_idpessoa', 'exe_debitopessoa_datainclusao',
       'exe_debitoboleto_iddebitoboleto', 'exe_debitoboleto_iddebito',
       'exe_debitoboleto_valororiginal', 'exe_debitoboleto_valorpago',
       'exe_debitoboleto_datapagamento',
       'exe_debito_multadiaria_iddebitomultadiaria',
       'exe_debito_multadiaria_iddebito',
       'exe_debito_multadiaria_valormultadiaria',
       'exe_debito_multadiaria_datainicio', 'exe_debito_multadiaria_datafinal',
       'exe_creditopagamento_idcreditopagamento',
       'exe_creditopagamento_iddebito', 'exe_creditopagamento_valorcredito',
       'exe_creditopagamento_datainclusao', 'exe_parcelam

In [15]:
# Debtor Profile Features
features_df['is_civil_servant'] = grouped['is_civil_servant'].max().astype(int)

In [16]:
features_df

,total_debt_value,avg_parcel_value,max_debt_value,total_fines_only,total_ressarcimento_only,has_daily_fine,total_paid_amount,percent_paid_amount,n_installments,n_broken_installments,n_reopened_installments,perc_broken_installments,perc_failed_installments,is_civil_servant
exe_debitopessoa_idpessoa,,,,,,,,,,,,,,
164,33809.5900,NaN,33809.59,0.0000,33809.59,0,0.00,0.000000,0,0,0,0.0,0.0,0
247,1800.0000,600.0000,600.00,1800.0000,0.00,0,1836.72,1.020400,0,0,0,0.0,0.0,0
288,10000.0000,781.2475,2500.00,10000.0000,0.00,0,0.00,0.000000,1,0,0,0.0,0.0,1
701,716.0928,55.0000,110.00,716.0928,0.00,0,612.00,0.854638,0,0,0,0.0,0.0,0
773,5000.0000,500.0000,500.00,5000.0000,0.00,0,5539.16,1.107832,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76188,415.3500,NaN,415.35,0.0000,415.35,0,0.00,0.000000,0,0,0,0.0,0.0,0
76662,63887.4800,6388.7500,31943.73,31943.7500,31943.73,0,0.00,0.000000,0,0,0,0.0,0.0,0
76930,0.0000,NaN,0.00,0.0000,0.00,0,0.00,NaN,0,0,0,0.0,0.0,0


In [28]:
# Legal Process Features
features_df['n_processos'] = grouped['processo_idprocesso'].nunique()
features_df['n_transitado_em_julgado'] = grouped['processo_data_transito_julgado'].apply(lambda x: x.notna().sum())
features_df['n_protests'] = grouped['protesto_titulosremessa_idtituloremessa'].nunique()

features_df['n_failed_protests'] = grouped['protesto_status'].apply(lambda x: x.isin([8, 9, 10, 11, 12]).sum())

features_df['perc_failed_protests'] = features_df['n_failed_protests'] / features_df['n_protests']
features_df['perc_failed_protests'] = features_df['perc_failed_protests'].fillna(0)

features_df['n_pge_handovers'] = grouped['pge_processo_idprocessopge'].nunique()

In [30]:
features_df

,total_debt_value,avg_parcel_value,max_debt_value,total_fines_only,total_ressarcimento_only,has_daily_fine,total_paid_amount,percent_paid_amount,n_installments,n_broken_installments,n_reopened_installments,perc_broken_installments,perc_failed_installments,is_civil_servant,n_processos,n_transitado_em_julgado,n_protests,n_failed_protests,perc_failed_protests,n_pge_handovers
exe_debitopessoa_idpessoa,,,,,,,,,,,,,,,,,,,,
164,33809.5900,NaN,33809.59,0.0000,33809.59,0,0.00,0.000000,0,0,0,0.0,0.0,0,1,1,0,0,0.0,0
247,1800.0000,600.0000,600.00,1800.0000,0.00,0,1836.72,1.020400,0,0,0,0.0,0.0,0,1,0,0,0,0.0,0
288,10000.0000,781.2475,2500.00,10000.0000,0.00,0,0.00,0.000000,1,0,0,0.0,0.0,1,1,4,0,0,0.0,0
701,716.0928,55.0000,110.00,716.0928,0.00,0,612.00,0.854638,0,0,0,0.0,0.0,0,4,14,0,0,0.0,0
773,5000.0000,500.0000,500.00,5000.0000,0.00,0,5539.16,1.107832,0,0,0,0.0,0.0,0,3,10,2,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76188,415.3500,NaN,415.35,0.0000,415.35,0,0.00,0.000000,0,0,0,0.0,0.0,0,1,0,0,0,0.0,0
76662,63887.4800,6388.7500,31943.73,31943.7500,31943.73,0,0.00,0.000000,0,0,0,0.0,0.0,0,1,0,0,0,0.0,0
76930,0.0000,NaN,0.00,0.0000,0.00,0,0.00,NaN,0,0,0,0.0,0.0,0,1,0,0,0,0.0,0


In [31]:
features_df.to_csv('../data/public/raw/tcern_debtors_features.csv', index=False)